In [ ]:
# Third-party
from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u
from astropy.table import Table
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from pyia import GaiaData

In [ ]:
gc_frame = coord.Galactocentric(z_sun=0*u.pc, galcen_distance=8.3*u.kpc)

In [ ]:
# see FGK-select.ipynb
stype = 'fgk'
# stype = 'af'

if stype == 'af':
    vmax = 1E2
    hex_h = 150 # pc
    
elif stype == 'fgk':
    vmax = 3e2
    hex_h = 120

g = GaiaData('../data/{0}.fits'.format(stype))

c = g.skycoord
galcen = c.transform_to(gc_frame)

In [ ]:
gal = c.galactic
gal.set_representation_cls('cartesian')

---

## Hexagons

In [ ]:
def get_hexagons(h):
    a = np.sqrt(3)/2 * h # inner side
    
    pas = dict() # keyed by "ring"
    pas[0] = list()
    pas[1] = list()
    pas[2] = list()
    
    pa0 = mpl.patches.RegularPolygon((0., 0.), numVertices=6, 
                                     radius=h, orientation=np.pi/2)
    pas[0].append(pa0.get_verts())

    for ang in np.arange(0, 360, 60)*u.deg:
        # Ring 1
        xy0 = 2*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[1].append(pa.get_verts())
        
        # Ring 2
        xy0 = 4*a * np.array([np.cos(ang+90*u.deg), np.sin(ang+90*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[2].append(pa.get_verts())
        
        xy0 = 3*h * np.array([np.cos(ang+120*u.deg), np.sin(ang+120*u.deg)])
        pa = mpl.patches.RegularPolygon(xy0, numVertices=6, 
                                        radius=h, orientation=np.pi/2)
        pas[2].append(pa.get_verts())
        
    return pas

In [ ]:
hexs = get_hexagons(hex_h)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

for k in hexs.keys():
    for pa in hexs[k]:
        pa = mpl.patches.Polygon(pa, facecolor='none', edgecolor='#333333')
        ax.add_patch(pa)

        
ax.plot(gal.u, gal.v, 
        marker=',', alpha=0.1, color='k',
        linestyle='none', zorder=100)
        
lim = 1000
ax.set_xlim(-lim, lim)
ax.set_ylim(-lim, lim)

ax.set_xlabel('$x$ [pc]')
ax.set_ylabel('$y$ [pc]')

---

In [ ]:
cyl = gal.transform_to(gc_frame)
cyl.set_representation_cls('cylindrical')

xyz = np.vstack((gal.u.to(u.pc).value, 
                 gal.v.to(u.pc).value,
                 gal.w.to(u.pc).value)).T

# UVW = np.vstack((gal_fgk.U.to(u.km/u.s).value, 
#                  gal_fgk.V.to(u.km/u.s).value, 
#                  gal_fgk.W.to(u.km/u.s).value)).T

UVW = np.vstack((cyl.d_rho.to(u.km/u.s).value, 
                 - ((cyl.rho * cyl.d_phi).to(u.km/u.s, u.dimensionless_angles()).value + 220.), 
                 cyl.d_z.to(u.km/u.s).value)).T

In [ ]:
for k in hexs.keys():
    for b, pa in enumerate(hexs[k]):
        hex_mask = mpl.patches.Path(pa).contains_points(xyz[:, :2])
        print(hex_mask.sum())

        lim = 150
        bins = np.linspace(-lim, lim, 101)

        fig, axes = plt.subplots(2, 2, figsize=(8, 7.2))
        
        for a, (i, j) in zip([0, 2, 3], [(0, 1), (0, 2), (1, 2)]):
            ax = axes.flat[a]
            H, xe, ye = np.histogram2d(UVW[hex_mask,i], UVW[hex_mask,j], bins=bins)
            ax.pcolormesh(xe, ye, H.T, 
                          norm=mpl.colors.LogNorm(), 
                          cmap='magma', vmin=1, vmax=vmax)
            ax.set_xlim(-lim, lim)
            ax.set_ylim(-lim, lim)
        
        axes[0, 0].set_ylabel('$-v_\phi-220$')
        axes[1, 0].set_ylabel('$v_z$')
        axes[1, 0].set_xlabel('$v_R$')
        axes[1, 1].set_xlabel('$-v_\phi-220$')
        axes[0, 0].xaxis.set_ticklabels([])
        axes[1, 1].yaxis.set_ticklabels([])
        
        # axes[0,1].set_visible(False)
        ax = axes[0,1]
        for k_ in hexs.keys():
            for pa_ in hexs[k_]:
                pa_ = mpl.patches.Polygon(pa_, facecolor='none', edgecolor='#333333')
                ax.add_patch(pa_)
                
        ax.add_patch(mpl.patches.Polygon(pa, facecolor='#333333', edgecolor='#333333'))
        ax.set_xlim(-750, 750)
        ax.set_ylim(-750, 750)
        ax.set_xlabel('$x$ [pc]')
        ax.set_ylabel('$y$ [pc]')

        fig.tight_layout()
        fig.savefig('../plots/uvw/{2}-{0}-{1:02d}.png'.format(k, b, stype), dpi=250)
        plt.close(fig)

```
convert -delay 30 -loop 0 1-*.png ring1.gif
convert -delay 30 -loop 0 2-*.png ring2.gif
convert -delay 30 -loop 1 *.png all.gif
```

In [ ]:
line_cmd = 'convert -delay 30 -loop 0 {0} full-line{1}-{2}.gif'
for k, a in enumerate([[(2,0),(1,0),(0,0),(1,3),(2,6)],
                       [(2,2),(1,1),(0,0),(1,4),(2,8)],
                       [(2,4),(1,2),(0,0),(1,5),(2,10)]]):
    names = ['{2}-{0}-{1:02d}.png'.format(i, j, stype) for i, j in a] 
    print(line_cmd.format(' '.join(names + names[::-1][1:-1]), k, stype))